In [24]:
import os
from six.moves import urllib

import matplotlib.pyplot as plt
# import mediapy as media
import numpy as np
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

from official.vision.beta.configs import video_classification
from official.vision.beta.projects.movinet.configs import movinet as movinet_configs
from official.vision.beta.projects.movinet.modeling import movinet
from official.vision.beta.projects.movinet.modeling import movinet_layers
from official.vision.beta.projects.movinet.modeling import movinet_model

In [23]:
!pip install tensorflow_addons

     |████████████████████████████████| 679 kB 3.9 MB/s 


In [21]:
%cd models


/content/models


In [4]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 59640, done.
remote: Counting objects: 100% (1120/1120), done.
remote: Compressing objects: 100% (442/442), done.
remote: Total 59640 (delta 753), reused 994 (delta 659), pack-reused 58520
Receiving objects: 100% (59640/59640), 573.74 MiB | 23.04 MiB/s, done.
Resolving deltas: 100% (41385/41385), done.


In [25]:
model_id = 'a0'
batch_size = 8
resolution = 172
num_frames = 20
num_classes = 2
tf.keras.backend.clear_session()

backbone = movinet.Movinet(
    model_id=model_id)
model = movinet_model.MovinetClassifier(
    backbone=backbone,
    num_classes=600)
model.build([batch_size, num_frames, resolution, resolution, 3])

# Load pretrained weights from TF Hub
movinet_hub_url = f'https://tfhub.dev/tensorflow/movinet/{model_id}/base/kinetics-600/classification/1'
movinet_hub_model = hub.KerasLayer(movinet_hub_url, trainable=True)
pretrained_weights = {w.name: w for w in movinet_hub_model.weights}
model_weights = {w.name: w for w in model.weights}
for name in pretrained_weights:
  model_weights[name].assign(pretrained_weights[name])

# Wrap the backbone with a new classifier to create a new classifier head
# with num_classes outputs
model = movinet_model.MovinetClassifier(
    backbone=backbone,
    num_classes=num_classes)
model.build([batch_size, num_frames, resolution, resolution, 3])

# Freeze all layers except for the final classifier head
for layer in model.layers[:-1]:
  layer.trainable = False
model.layers[-1].trainable = True

In [26]:
model.summary()

Model: "movinet_classifier_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, None, None, None, 0         
_________________________________________________________________
movinet (Movinet)            ({'stem': (None, None, No 911583    
_________________________________________________________________
classifier_head_1 (Classifie (None, 2)                 989186    
Total params: 1,900,769
Trainable params: 989,186
Non-trainable params: 911,583
_________________________________________________________________


In [27]:
!python -c "import tensorflow as tf; print(tf.__version__)"

2021-08-03 18:29:49.910786: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2.5.0
